In [1]:
import pandas as pd

pd.set_option('display.max_rows', 10)

# Create an empty DataFrame with the specified columns
df = pd.read_csv("data.csv", index_col=0)

# Show the empty DataFrame
data = df


In [2]:
# Replace NaN values with 0
data_filled = data.fillna(0)

# Ensure all columns except 'replicates' (since it might contain non-numeric values originally) are numeric
data_filled = data_filled.apply(pd.to_numeric, errors='ignore')

# Display the data types to confirm changes
data_filled.dtypes, data_filled.head()

data_filled


,round,split,iter,replicates,duplicates,distortion,m1,m2,m3
0,-1,0.0,10k,0,8.0,1,1.000000,0.745803,0.854396
1,-1,0.0,10k,0,100.0,1,0.997500,0.802817,0.889632
2,-1,0.0,10k,0,0.1,0,0.967742,0.952381,0.960000
3,-1,0.0,10k,0,1.0,0,1.000000,0.919192,0.957895
4,-1,0.0,10k,0,0.1,1,1.000000,0.766667,0.867924
...,...,...,...,...,...,...,...,...,...
3839,-12,0.0,50k,_20,100.0,0,1.000000,0.921762,0.959288
3840,-12,0.0,50k,_20,0.1,0,0.983607,0.952381,0.967742
3841,-12,0.0,50k,_20,0.1,1,1.000000,0.766667,0.867924
3842,-12,0.0,50k,_20,8.0,1,0.997895,0.757794,0.861427


In [3]:
# Remove "_" from the 'replicates' column and convert to numeric if possible
data_filled['replicates'] = data_filled['replicates'].astype(str).str.replace("_", "")
data_filled['replicates'] = pd.to_numeric(data_filled['replicates'], errors='coerce').fillna(0)

# Make 'round' and 'split' values positive
data_filled['round'] = data_filled['round'].abs()
data_filled['split'] = data_filled['split'].abs()


# Rename columns 'm1', 'm2', 'm3' to 'precision', 'recall', 'f1score'
data_renamed = data_filled.rename(columns={'m1': 'precision', 'm2': 'recall', 'm3': 'f1score'})

# Display the first few rows to confirm the column names have been updated
data_renamed


,round,split,iter,replicates,duplicates,distortion,precision,recall,f1score
0,1,0.0,10k,0,8.0,1,1.000000,0.745803,0.854396
1,1,0.0,10k,0,100.0,1,0.997500,0.802817,0.889632
2,1,0.0,10k,0,0.1,0,0.967742,0.952381,0.960000
3,1,0.0,10k,0,1.0,0,1.000000,0.919192,0.957895
4,1,0.0,10k,0,0.1,1,1.000000,0.766667,0.867924
...,...,...,...,...,...,...,...,...,...
3839,12,0.0,50k,20,100.0,0,1.000000,0.921762,0.959288
3840,12,0.0,50k,20,0.1,0,0.983607,0.952381,0.967742
3841,12,0.0,50k,20,0.1,1,1.000000,0.766667,0.867924
3842,12,0.0,50k,20,8.0,1,0.997895,0.757794,0.861427


In [4]:
# Define a function to map 'round' values to model names
def map_round_to_model(round_number):
    if 1 <= round_number <= 3:
        return "Both"
    elif 4 <= round_number <= 6:
        return "No Diri"
    elif 7 <= round_number <= 9:
        return "None"
    elif 10 <= round_number <= 12:
        return "No Empirical"
    else:
        return "Unknown"  # For any round numbers outside the specified ranges

# Apply the mapping function to the 'round' column to create the new 'model' column
data_renamed['model'] = data_renamed['round'].apply(map_round_to_model)



# Define a function to map 'round' values to prior categories
def map_round_to_prior(round_number):
    if round_number in [1, 4, 7, 10]:
        return "Pitman"
    elif round_number in [2, 5, 8, 11]:
        return "uniform"
    elif round_number in [3, 6, 9, 12]:
        return "Bounded NBD"
    else:
        return "Unknown"  # For any round numbers outside the specified or considered ranges

# Apply the mapping function to the 'round' column to create the new 'prior' column
data_renamed['prior'] = data_renamed['round'].apply(map_round_to_prior)

# Define a function to map 'duplicates' values to duplicates level categories
def map_duplicates_to_level(duplicates_value):
    if duplicates_value == 0.1:
        return "Low"
    elif duplicates_value == 1.0:
        return "Medium"
    elif duplicates_value == 8.0:
        return "High"
    elif duplicates_value == 100.0:
        return "Very High"
    else:
        return "Unknown"  # For any duplicates values outside the specified ranges

# Apply the mapping function to the 'duplicates' column to create the new 'duplicates_level' column
data_renamed['duplicates_level'] = data_renamed['duplicates'].apply(map_duplicates_to_level)

# Define a function to map 'distortion' values to distortion level categories
def map_distortion_to_level(distortion_value):
    if distortion_value == 0:
        return "Low"
    elif distortion_value == 1:
        return "High"
    else:
        return "Unknown"  # For any distortion values outside the specified values

# Apply the mapping function to the 'distortion' column to create the new 'distortion_level' column
data_renamed['distortion_level'] = data_renamed['distortion'].apply(map_distortion_to_level)

def map_iter_to_num(iter):
    if iter == '10k':
        return 10000
    elif iter == '50k':
        return 50000
    else:
        return 0  # For any distortion values outside the specified values

# Apply the mapping function to the 'distortion' column to create the new 'distortion_level' column
data_renamed['num_iter'] = data_renamed['iter'].apply(map_iter_to_num)

# Select only the specified columns
data_selected_columns = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates']]

# Select and reorder the dataset with only the specified columns
data_final = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter',
                           'replicates', 'precision', 'recall', 'f1score']]

# Display the first few rows to confirm the changes
data_final






,model,prior,duplicates_level,distortion_level,num_iter,replicates,precision,recall,f1score
0,Both,Pitman,High,High,10000,0,1.000000,0.745803,0.854396
1,Both,Pitman,Very High,High,10000,0,0.997500,0.802817,0.889632
2,Both,Pitman,Low,Low,10000,0,0.967742,0.952381,0.960000
3,Both,Pitman,Medium,Low,10000,0,1.000000,0.919192,0.957895
4,Both,Pitman,Low,High,10000,0,1.000000,0.766667,0.867924
...,...,...,...,...,...,...,...,...,...
3839,No Empirical,Bounded NBD,Very High,Low,50000,20,1.000000,0.921762,0.959288
3840,No Empirical,Bounded NBD,Low,Low,50000,20,0.983607,0.952381,0.967742
3841,No Empirical,Bounded NBD,Low,High,50000,20,1.000000,0.766667,0.867924
3842,No Empirical,Bounded NBD,High,High,50000,20,0.997895,0.757794,0.861427


In [20]:
# Finding duplicates and keeping them to show
duplicates_to_show = data_cleaned[data_cleaned.duplicated(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'], keep=False)]

# Sorting the duplicates for better readability
duplicates_sorted = duplicates_to_show.sort_values(by=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'])

duplicates_sorted


,model,prior,duplicates_level,distortion_level,num_iter,replicates,precision,recall,f1score
2291,Both,Bounded NBD,Low,High,50000,7,1.000000,0.750000,0.857143
2292,Both,Bounded NBD,Low,High,50000,7,1.000000,0.733333,0.846154
2769,No Diri,Bounded NBD,High,Low,50000,6,1.000000,0.943291,0.970818
2771,No Diri,Bounded NBD,High,Low,50000,6,0.002605,1.000000,0.005196
2773,No Diri,Bounded NBD,Low,Low,50000,6,0.000121,1.000000,0.000241
...,...,...,...,...,...,...,...,...,...
3213,None,Bounded NBD,Medium,Low,50000,2,1.000000,0.929293,0.963351
3210,None,Bounded NBD,Very High,High,50000,2,0.997610,0.839705,0.911872
3218,None,Bounded NBD,Very High,High,50000,2,0.002955,1.000000,0.005893
3207,None,Bounded NBD,Very High,Low,50000,2,0.002898,1.000000,0.005780


In [21]:
# Checking for missing combinations in the first 6 columns
# First, we create a DataFrame of all possible combinations of these columns

# Get unique values for each of the first 6 columns
model_values = data_cleaned['model'].unique()
prior_values = data_cleaned['prior'].unique()
duplicates_level_values = data_cleaned['duplicates_level'].unique()
distortion_level_values = data_cleaned['distortion_level'].unique()
num_iter_values = data_cleaned['num_iter'].unique()
replicates_values = data_cleaned['replicates'].unique()

# Use itertools.product to create all possible combinations
from itertools import product
all_combinations = list(product(model_values, prior_values, duplicates_level_values, distortion_level_values, num_iter_values, replicates_values))

# Convert to DataFrame
all_combinations_df = pd.DataFrame(all_combinations, columns=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'])

# Now, find missing combinations by checking which rows in all_combinations_df are not present in data_cleaned_no_duplicates
missing_combinations = pd.merge(all_combinations_df, data_cleaned_no_duplicates, on=['model', 'prior', 'duplicates_level', 'distortion_level', 'num_iter', 'replicates'], how='left', indicator=True).query('_merge == "left_only"').drop(columns=['precision', 'recall', 'f1score', '_merge'])

missing_combinations_count = missing_combinations.shape[0]

print(missing_combinations_count)
missing_combinations


25


,model,prior,duplicates_level,distortion_level,num_iter,replicates
831,Both,Bounded NBD,Low,High,50000,4
1949,None,Pitman,High,High,50000,17
1989,None,Pitman,High,Low,50000,17
2029,None,Pitman,Very High,High,50000,17
2069,None,Pitman,Very High,Low,50000,17
...,...,...,...,...,...,...
2719,None,Bounded NBD,Very High,Low,50000,20
2759,None,Bounded NBD,Low,High,50000,20
2799,None,Bounded NBD,Low,Low,50000,20
2839,None,Bounded NBD,Medium,High,50000,20


In [6]:
# Group by the specified columns and compute mean and standard deviation for precision, recall, and f1score
grouped_data = data_final.groupby(['model', 'prior', 'duplicates_level', 'distortion_level'])

# Compute mean
mean_values = grouped_data[['precision', 'recall', 'f1score']].mean().reset_index()

# Compute standard deviation
std_dev_values = grouped_data[['precision', 'recall', 'f1score']].std().reset_index()

# Rename columns for clarity
mean_values.rename(columns={
    'precision': 'precision_mean', 
    'recall': 'recall_mean', 
    'f1score': 'f1score_mean'}, inplace=True)

std_dev_values.rename(columns={
    'precision': 'precision_std', 
    'recall': 'recall_std', 
    'f1score': 'f1score_std'}, inplace=True)

# Display the mean values
mean_values.head(), std_dev_values.head()


(  model        prior duplicates_level distortion_level  precision_mean  \
 0  Both  Bounded NBD             High             High        0.998965   
 1  Both  Bounded NBD             High              Low        0.999936   
 2  Both  Bounded NBD              Low             High        1.000000   
 3  Both  Bounded NBD              Low              Low        0.982430   
 4  Both  Bounded NBD           Medium             High        0.987646   
 
    recall_mean  f1score_mean  
 0     0.729856      0.842993  
 1     0.938099      0.968029  
 2     0.746250      0.854624  
 3     0.952381      0.967164  
 4     0.749743      0.852176  ,
   model        prior duplicates_level distortion_level  precision_std  \
 0  Both  Bounded NBD             High             High       0.001345   
 1  Both  Bounded NBD             High              Low       0.000403   
 2  Both  Bounded NBD              Low             High       0.000000   
 3  Both  Bounded NBD              Low              Low    

In [7]:
# Merge mean and standard deviation dataframes on the groupby columns
combined_data = pd.merge(mean_values, std_dev_values, 
                         on=['model', 'prior', 'duplicates_level', 'distortion_level'])


pd.set_option('display.max_rows', None)


# Display the combined dataframe
combined_data


,model,prior,duplicates_level,distortion_level,precision_mean,recall_mean,f1score_mean,precision_std,recall_std,f1score_std
0,Both,Bounded NBD,High,High,0.998965,0.729856,0.842993,0.001345,0.035021,0.023079
1,Both,Bounded NBD,High,Low,0.999936,0.938099,0.968029,0.000403,0.002825,0.001528
2,Both,Bounded NBD,Low,High,1.000000,0.746250,0.854624,0.000000,0.013337,0.008682
3,Both,Bounded NBD,Low,Low,0.982430,0.952381,0.967164,0.005593,0.000000,0.002719
4,Both,Bounded NBD,Medium,High,0.987646,0.749743,0.852176,0.005473,0.022038,0.012281
5,Both,Bounded NBD,Medium,Low,0.999728,0.927121,0.962049,0.000880,0.005343,0.002941
6,Both,Bounded NBD,Very High,High,0.998112,0.808249,0.892957,0.001706,0.026935,0.016282
7,Both,Bounded NBD,Very High,Low,1.000000,0.922370,0.959617,0.000000,0.001025,0.000555
8,Both,Pitman,High,High,0.999497,0.714269,0.832683,0.001271,0.034298,0.022997
9,Both,Pitman,High,Low,1.000000,0.938738,0.968399,0.000000,0.002720,0.001447


In [8]:
pd.set_option('display.max_rows', 10)

In [9]:
combined_data.to_csv("clean_data.csv")